subcellular loc data

In [25]:
import pandas as pd 
import matplotlib.pyplot as plt
from pylab import *
df1 = pd.read_table("sequences_info/All RNA subcellular localization data.txt", low_memory=False)
df1.drop(['RNALocate_ID','Gene_Name', 'Gene_symbol','PMID', 'Description'],axis=1,inplace=True)
df1[(df1['RNA_category']=='lncRNA') & (df1['Species']=='Homo sapiens')]
df1

,Gene_ID,RNA_category,Species,SubCellular_Localization
0,NCBI:20983,mRNA,Mus musculus,Nucleus
1,NCBI:20650,mRNA,Mus musculus,Nucleus
2,NCBI:20648,mRNA,Mus musculus,Cytoplasm
3,NCBI:20873,mRNA,Mus musculus,Nucleus
4,NCBI:17764,mRNA,Mus musculus,Cytoplasm
...,...,...,...,...
213424,NaN,Y RNA,Homo sapiens,Microvesicle
213425,NaN,Y RNA,Homo sapiens,Microvesicle
213426,NaN,Y RNA,Homo sapiens,Microvesicle
213427,NaN,Y RNA,Homo sapiens,Microvesicle


lncRNA sequence

In [26]:
df2 = pd.read_table("sequences_info/human_RNA_sequence.txt")
df2.drop('Refseq_ID',axis=1,inplace=True)
df2.shape

(133905, 2)

merge

In [27]:
df = pd.merge(df1,df2,how='inner',on='Gene_ID')
df_lnc = df[df['RNA_category']=='lncRNA']
df_lnc = df_lnc.drop_duplicates(subset=['Gene_ID'], keep="first")
df_lnc=df_lnc.reset_index(drop=True)
df_lnc.info

<bound method DataFrame.info of              Gene_ID RNA_category       Species SubCellular_Localization  \
0     NCBI:100462981       lncRNA  Homo sapiens                  Exosome   
1         NCBI:10057       lncRNA  Homo sapiens              Nucleoplasm   
2          NCBI:1031       lncRNA  Homo sapiens                  Cytosol   
3          NCBI:2177       lncRNA  Homo sapiens                  Cytosol   
4        NCBI:221178       lncRNA  Homo sapiens                  Nucleus   
...              ...          ...           ...                      ...   
3439      NCBI:94158       lncRNA  Homo sapiens                  Exosome   
3440       NCBI:9597       lncRNA  Homo sapiens                  Exosome   
3441      NCBI:96597       lncRNA  Homo sapiens                  Exosome   
3442       NCBI:9808       lncRNA  Homo sapiens                  Exosome   
3443       NCBI:9834       lncRNA  Homo sapiens                  Exosome   

                                               Sequence

In [28]:
df_lnc['SubCellular_Localization'].value_counts()

SubCellular_Localization
Exosome                  2442
Chromatin                 356
Nucleus                   225
Cytoplasm                 111
Cytosol                   105
Nucleoplasm                99
Nucleolus                  70
Membrane                   25
Insoluble cytoplasm         6
Nuclear                     2
Endoplasmic reticulum       1
Microsome                   1
Soma                        1
Name: count, dtype: int64

generate fasta type

In [29]:
with open("sequences_info/lnc.fasta","w") as f:
    for i in range(len(df_lnc.Sequence)):
        f.write('>'+str(i)+'\n')
        f.write(df_lnc.Sequence.iloc[i]+'\n')

Use CD-HIT, with cutoff 0.8

In [30]:
with open('cdhit_results/lnc_80') as fa:
    lnc_index = []
    for line in fa:
        # 去除末尾换行符
        line = line.replace('\n','')
        if line.startswith('>'):
            # 去除 > 号
            seq_index = line[1:]
            lnc_index.append(int(seq_index))

In [31]:
df_lnc = df_lnc.iloc[lnc_index]
df_lnc = df_lnc.reset_index(drop=True) # update index

In [32]:
def labelize(loc):
    if loc == "Exosome":
        return 0
    if loc == "Chromatin":
        return 1
    if (loc == "Cytoplasm") or (loc == "Insoluble cytoplasm"):
        return 2
    if loc == "Cytosol":
        return 3
    if (loc == "Nucleus") or (loc == "Nuclear") or (loc == "Nucleolus") or (loc == "Nucleoplasm"):
        return 4

In [33]:
df_lnc['SubCellular_Localization'].value_counts()

SubCellular_Localization
Exosome                  2326
Chromatin                 353
Nucleus                   224
Cytoplasm                 110
Cytosol                   105
Nucleoplasm                98
Nucleolus                  70
Membrane                   25
Insoluble cytoplasm         6
Nuclear                     2
Endoplasmic reticulum       1
Microsome                   1
Soma                        1
Name: count, dtype: int64

In [34]:
df_lnc=df_lnc[df_lnc['SubCellular_Localization'].isin(['Exosome','Nucleus','Cytosol','Chromatin','Cytoplasm','Nucleoplasm', 'Nucleolus','Nuclear','Insoluble cytoplasm'])]
df_lnc['SubCellular_Localization'].value_counts()

SubCellular_Localization
Exosome                2326
Chromatin               353
Nucleus                 224
Cytoplasm               110
Cytosol                 105
Nucleoplasm              98
Nucleolus                70
Insoluble cytoplasm       6
Nuclear                   2
Name: count, dtype: int64

In [35]:
df_lnc.drop(['Gene_ID','RNA_category','Species'], axis=1)
order=['Sequence','SubCellular_Localization']
df_lnc=df_lnc[order].reset_index(drop=True)

In [36]:
df_lnc['SubCellular_Localization']=df_lnc.apply(lambda x: labelize(x.SubCellular_Localization),axis=1)
df_lnc.columns=['sequence','label']


In [37]:
df_lnc['label'].value_counts()

label
0    2326
4     394
1     353
2     116
3     105
Name: count, dtype: int64

In [38]:
df_lnc['sequence_length'] = df_lnc['sequence'].str.len()
print(df_lnc['sequence_length'].describe())

count     3294.000000
mean      1917.693078
std       2368.071184
min         61.000000
25%        768.250000
50%       1540.000000
75%       2445.750000
max      91671.000000
Name: sequence_length, dtype: float64


In [39]:
df_filtered = df_lnc[df_lnc['sequence_length'] <= 10000]
df_final = df_filtered.drop(columns='sequence_length')
df_final['label'].value_counts()
df_final

,sequence,label
0,ACAACCTTAACCAAACCATTTACTCAAATAAAGTATAGGCGATAGA...,0
1,ATGCTCGGGAGCGTGGTTGAGCGGCTGGCGCGGTTGTCCTGGAGCA...,4
2,AGATTAACCATCCCAGTCCTTCTGTCAGTCTCCGATGCCATCATGC...,3
3,GGGAAAGTCGAAAACTACGGGCGGCGACGGCTTCTCGGAAGTAATT...,3
4,GGCGCGCACCCGGGACAGTTTCCAGTCGGCTTCTCCGTGACCTTCC...,4
...,...,...
3289,CGCTCAAGAGAGCAAGATGTTCTCGTGTTGCTTCTCCACTTTGAGA...,0
3290,TGATCTGAAAGGAATGGAAGCACAAAATGATGAATAAGGTATTTTT...,0
3291,GAAAACGAAGCCCCCTCCTGTATTGACAGCTGGGAATTGTGGAGTC...,0
3292,AATCATCTCATCTCAGATGCCTGGCAACATCTAATTCCTCAGCTCC...,0


In [43]:
df_final.to_csv('../dataset/dataset_lnc.txt',sep='\t',index=False)